In [ ]:

import os
import sys
import seaborn as sns

# import pyLDAvis
import pyLDAvis.gensim_models

import numpy as np

import pandas as pd
from gensim import corpora, models
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# converting all json files for all-community-building channel to dataframe using slack_parser method from loader script
df_community = sl.slack_parser('../Anonymized_B6SlackExport_25Nov23/anonymized/all-community-building/')

In [ ]:
# Convert 'message_sent_time' to datetime format
# df['message_sent_time'] = pd.to_datetime(df['message_sent_time'])
df_community['msg_sent_time'] = pd.to_datetime(df_community['msg_sent_time'], unit='s')

# Sort the DataFrame by 'message_sent_time'
df_community = df_community.sort_values(by='msg_sent_time')

# Calculate time differences between consecutive messages
df_community['time_difference'] = df_community['msg_sent_time'].diff()

# Plot the distribution of time differences
plt.hist(df_community['time_difference'].dt.total_seconds(), bins=50, edgecolor='black')
plt.xlabel('Time Differences (seconds)')
plt.ylabel('Frequency')
plt.title('Distribution of Time Differences Between Messages')
plt.show()


In [ ]:
### Plot a histogram of the time difference between Consecutive replies

In [ ]:
# Find the index of the maximum reply_count
max_reply_count= df_community['reply_count'].max()
max_reply_count

In [ ]:
msg_content = sl.get_channel_messages(channel_name='all-community-building')

In [ ]:
def filter_msg(msg_content):
    
    for msg in msg_content:
        if 'reply_count' in msg.keys():
            if msg['reply_count']==75:
                return msg

In [ ]:
top_reply_count_message = filter_msg(msg_content)

In [ ]:
top_reply_count_message = filter_msg(msg_content)

In [ ]:
df_replies.columns

In [ ]:
df_replies['ts'] = pd.to_datetime(df_replies['ts'], unit='s')

# Sort the DataFrame by 'message_sent_time'
df_replies = df_replies.sort_values(by='ts')

# Calculate time differences between consecutive messages
df_replies['time_difference'] = df_replies['ts'].diff()

# Plot the distribution of time differences
plt.hist(df_replies['time_difference'].dt.total_seconds(), bins=50, edgecolor='black')
plt.xlabel('Time Differences (seconds)')
plt.ylabel('Frequency')
plt.title('Distribution of Time Differences Between Messages')
plt.show()


In [ ]:
!pip3 install gensim
!pip3 install pyLDAvis

In [ ]:
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
df_community.head()

In [ ]:
preprocess = Preprocessing(df_community)

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

# clean text 
df_community = preprocess.cleantext('msg_content', 'clean_msg_content')

# Stem words
df_community = preprocess.stem('clean_msg_content')

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# Used stemmer instead of lemmatizer
cleanKagslacklist = preprocess.filterSlackList(df_community['clean_msg_content'])

In [ ]:
cleanKagslacklist[1]

In [ ]:
def makeDict(mySlackList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(mySlackList)

def makeCorpus(mySlackList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(slackmessage) for slackmessage in mySlackList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [ ]:
"""Create model objects"""
kagDict   = makeDict(cleanKagslacklist)
kagCorpus = makeCorpus(cleanKagslacklist, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'kaggleLDAmodel')
corpora.MmCorpus.serialize('kaggleCorpus.mm', kagCorpus)
kagDict.save('kaggleDictionary.dict')

In [ ]:
kagLda = SaveLoad.load('kaggleLDAmodel')
kagDict = corpora.Dictionary.load('kaggleDictionary.dict')
kagCorpus = corpora.MmCorpus('kaggleCorpus.mm')

In [ ]:
kagLda.print_topics()

In [ ]:
num_topics = 10

for topic_id, topic in kagLda.show_topics(num_topics=num_topics, formatted=False):
    print(f"Topic {topic_id + 1}: {', '.join([word[0] for word in topic])}")


In [ ]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [ ]:
newIdx = translateLdaIdx(kagLda,ldaViz)

In [ ]:
def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [ ]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)
def sortByTopicToIdx(cleanKagslacklist,mySortedDf,myTopic,myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    srtIdx = list(mySortedDf[mySortedDf[myTopic]>myTopicThresh].index)
    return srtIdx

def sortSlackByIdx(cleanKagslacklist,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanKagslacklist)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [ ]:

sortedIdx = sortByTopicToIdx(cleanKagslacklist,kagDf,'topic2',myTopicThresh=0.1)# Word cloud for topic 
def makeWordCloud(cleanKagslacklist,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanKagslacklist,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedSlack = sortSlackByIdx(cleanKagslacklist,sortedIdx)
    filteredWords = ' '.join([' '.join(filter(None, string)) for string in mySortedSlack])
    myTopicCloud = WordCloud(width=800, height=800, max_font_size=100, scale=8).generate(filteredWords)

    # Display the word cloud using matplotlib
    plt.figure(figsize=(10, 10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")  # Turn off axis labels
    plt.show()


In [ ]:
def makeWordCloud(cleanKagslacklist, mySortedDf, myTopic, myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanKagslacklist, mySortedDf, myTopic, myTopicThresh=0.1)
    mySortedSlack = sortSlackByIdx(cleanKagslacklist, sortedIdx)
    filteredWords = ' '.join([' '.join(filter(None, string)) for string in mySortedSlack])
    myTopicCloud = WordCloud(width=800, height=800, max_font_size=100, scale=8).generate(filteredWords)

    # Display the word cloud using matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(myTopicCloud)
    plt.axis("off")  # Turn off axis labels
    plt.show()

In [ ]:
makeWordCloud(cleanKagslacklist,kagDf,'topic2',myTopicThresh=0.2)